# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

## `julia` Implementation

We will implement those two wave functions as PEPS in `julia` using `TensorOperations`:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf, BenchmarkTools

First, we define the Honeycomb structure, and some useful functions on it:

In [ ]:
struct HoneycombLattice
    shape::String
    a::Real
    verts::Vector{Tuple{Int64, Tuple}}
    bonds::Vector{Tuple}
    ψ#::Vector{Tuple{Integer, Array}}

    function HoneycombLattice(shape::String, a::Real, shift::Tuple, n::Integer, bc::String)
        @assert shape ∈ ("pyramid", "flower", "bricks") "Invalid lattice shape"
        @assert a > 0 "Bond length `a` must be positive"
        @assert length(shift) == 2 "Lattice shift must be of length 2"
        @assert n > 0 "Lattice order `n` must be positive"
        @assert bc ∈ ("periodic", "open") "Invalid lattice boundary condition"

        # define verts location based on lattice shape
        if shape == "pyramid"
            dx = a * sin(π/6)
            dy = a * cos(π/6)
            
            locs = [(0.0, 0.0)]
            for (i,x) in enumerate(range(start=dx, length=n, step=a+dx))
                for y in range(-dy*i, stop=dy*i, length=i+1)
                    append!(locs, [(-x,y), (-x-a,y)])
                end
            end
        
            for (j, y) in enumerate(range(start=-dy*(n-1), stop=dy*(n-1), length=n))
                append!(locs, [(-n*(dx+a)-dx, y)])
            end
        elseif shape == "bricks"
            if n == 1
                locs = [(0.0, 0.0), (0.0, a), (2*a, 0.0), (a, a), (a, 0.0), (2*a, a)]
            else
                locs = [(0.0, 0.0), (0.0, a), (a, 2*a), (a, a)]
                for i in 1:n
                    if i%2==0
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,2*a),(2*a,2*a),(2*a,a)]])
                    else
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,0),(2*a,0),(2*a,a)]])
                    end
                end
            end
            
        elseif shape == "flower"
            
        end
            
        # add shift to vertex locations and define verts array
        verts = [(ind, loc .+ shift) for (ind, loc) in enumerate(locs)]
        
        # detect bonds
        bonds = detectbonds(verts, a, shape)

        # fill in tensors
        A = AKLT_PEPS()
        missing = detectmissing(verts, bonds)
        ψ = []
        for (vert, miss) in zip(verts, missing)
            (ind, loc) = vert
            if length(miss) == 0
                push!(ψ, A)
            elseif length(miss) == 1
                push!(ψ, A[:,:,:,1])
            elseif length(miss) == 2
                push!(ψ, A[:,:,1,1])
            elseif length(miss) == 3
                push!(ψ, A[:,1,1,1])
            else
                error("Invalid number of missing bonds from vertex ", ind)
            end
        end
        
        # create structure
        new(shape, a, verts, bonds, ψ)
    end
end

function detectbonds(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, a::Real, shape::String=nothing)
    bonds = []
    bond_count = ones(Int64, length(verts))
    for vert in verts
        (ind, loc) = vert
        for (ind_j, loc_j) in nearestNeighboors(verts[ind:end], vert, a)
            if shape =="bricks" && (ind_j == ind + 1 && ind % 3 == 1 && ind != 1)
                continue
            end
            push!(bonds, ((ind, bond_count[ind]), (ind_j, bond_count[ind_j])))
            bond_count[ind] += 1
            bond_count[ind_j] += 1
        end
    end
    return bonds
end

function countneighbors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, bonds::Vector{Any})
    c = 0
    vert_bonds = []
    (ind, loc) = vert
    for bond in bonds
        if ind ∈ bond
            c += 1
            push!(vert_bonds, bond)
        end
    end
    return c, vert_bonds
end

function nearestNeighboors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, a::Real)
    neighbors = []
    (ind, loc) = vert
    for (ind_i, loc_i) in verts
        d = sum((loc .- loc_i).^2)
        if abs(d - a) < 1e-1*a
            push!(neighbors, (ind_i, loc_i))
        end
    end
    return neighbors
end

function detectmissing(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, bonds::Vector{Any})
    missing = [[1,2,3] for i in 1:length(verts)]
    for bond in bonds
        for (vert_ind, bond_ind) in bond
            deleteat!(missing[vert_ind], findall(x->x==bond_ind,missing[vert_ind]))
        end
    end
    return missing
end

function showlattice(fig::Plots.Plot, lat::HoneycombLattice, dolabel::Bool)
    x = [loc[1] for (ind, loc) in lat.verts]
    y = [loc[2] for (ind, loc) in lat.verts]
    for bond in lat.bonds
        inds = [b[1] for b in bond]
        dx = [lat.verts[i][2][1] for i in inds]
        dy = [lat.verts[i][2][2] for i in inds]
        plot!(fig, dx, dy, label="", color="blue")
    end
    scatter!(fig, x, y, aspect_ratio=1, label="", color="red")

    if dolabel
        for (ind, loc) in lat.verts
            annotate!(fig, loc[1], loc[2], ind)
        end
    end
    return fig
end

The tensors are defined using the following function:

In [ ]:
function AKLT_PEPS()
    Θ = zeros(4,2,2,2)
    Σ = zeros(2,2)
    A = zeros(4,2,2,2)
    
    # define Σ the singlet matrix
    Σ[1,2] = 1/sqrt(2); Σ[2,1] = -1/sqrt(2);
    
    # define Θ the projection operator
    for a in 1:2
        for b in 1:2
            for c in 1:2
                Θ[1,a,b,c] = (a+b+c == 3)
                Θ[2,a,b,c] = (a+b+c == 4)/sqrt(3)
                Θ[3,a,b,c] = (a+b+c == 5)/sqrt(3)
                Θ[4,a,b,c] = (a+b+c == 6)
            end
        end
    end
    
    # define A the total tensor
    for σ in 1:4
        for b in 1:2
            A[σ,:,b,:] = Θ[σ,:,b,:]*Σ*sqrt(6/10)
        end
    end
    return A
end

For example, here are two possible lattices:

In [ ]:
lat = HoneycombLattice("bricks", 1, (5.0,1.0), 3, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

In [ ]:
lat = HoneycombLattice("pyramid", 1, (5.0,1.0), 2, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

Next, in order to calculate local expectation values of a specific site, we need to orthogonalize the PEPS onto the site. This is done by contracting bonds of sites and then split them again using SVD. We start by writing soem useful functions:

In [ ]:
function bondtensor(ψ::HoneycombLattice, i::Int64)
    
end